In [1]:
import librosa
import os
import os.path
import numpy as np
import numba
from numba import jit, cuda 
from tqdm import tqdm

C:\Users\Rassam\Anaconda3\envs\NLP\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
C:\Users\Rassam\Anaconda3\envs\NLP\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [2]:
search_for, rate = librosa.load("search/Undertow.mp3")

C:\Users\Rassam\Anaconda3\envs\NLP\lib\site-packages\librosa\core\audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


In [40]:
len(search_for)

5227449

In [41]:
rate

22050

In [29]:
class loadSongs():
        
    data_dict = {"target_data.npy": ["target", list()], 
                 "source_data.npy": ["source", list()]}
    
    def load_songs(self):
        
        for file in self.data_dict:
            
            print("Loading {} data".format(self.data_dict[file][0]))
            
            if os.path.exists(file):
                self.data_dict[file][1] = np.load(file, allow_pickle = True)
            
            else:
                for song in tqdm(os.listdir(self.data_dict[file][0])):
                    if song.endswith(".mp3"):
                        self.data_dict[file][1].append([song, 
                                    librosa.load(os.path.join(self.data_dict[file][0], song))])
                np.save(file, self.data_dict[file][1], allow_pickle = True)
        
        return self.data_dict["target_data.npy"][1], self.data_dict["source_data.npy"][1]
    

In [30]:
target, source = loadSongs().load_songs()
#songs = load_songs("here/")

Loading target data
Loading source data


In [5]:
import math
print(cuda.gpus)

<Managed Device 0>


In [6]:
# X threads in a block; all blocks combined should make up the length of the search song
# so one thread in a block must compare upto X entries

@cuda.jit
def cosine_similarity_kernel(A, B, dot_prod, norm_A, norm_B):
    
    """
    A/B = vectors
    dot_prod = vector to store the dot product of search and this
    norm_A = norm of A .. bruh
    norm_B = bleh
    """
    
    size = cuda.grid(1)
    if size < len(search):
        dot_prod[size] = A[size] * B[size]
        norm_A += A[size]**2
        norm_B += B[size]**2
        

In [20]:
def find_songs(source, target):

    max_comparison = len(search_for)
    threadsperblock = 128
    blockspergrid = math.ceil(max_comparison / threadsperblock)
    
    for song in target:
        
        target_freq = song[1][0]
        target_rate = song[1][1]
        
        for another_song in source:
            
            compare = len(another_song[1][0]) if len(another_song[1][0]) < len(target_freq) else len(target_freq)
            
            for i in range(len(target_freq)):
                norm_A = cuda.to_device(np.zeros(compare))
                norm_B = cuda.to_device(np.zeros(compare))
                dot_prod = cuda.to_device(np.zeros(compare))
                
                target_glob = cuda.to_device(target_freq)
                source_glob = cuda.to_device(another_song[1][0])
                
                cosine_similarity_kernel[blockspergrid, threadsperblock](target_glob, source_glob,
                                                                        dot_prod, norm_A, norm_B)
                norm_A = norm_A.copy_to_host()
                norm_B = norm_B.copy_to_host()
                dot_prod = dot_prod.copy_to_host()
                
                similarity = dot_prod/math.sqrt(sum(norm_A) * sum(norm_B))
                
                if similarity > 0.5:
                    print("{} matched with {}".format(another_song[0], song[0]))
        

In [44]:
for song in target:
    print(song[0])
    break

03. Shaped by Fire.mp3
